In [1]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np

In [29]:
# load dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data = test_data.drop("index",axis=1)
data = pd.concat([train_data,test_data],sort=False)

In [30]:
data.shape

(258210, 14)

In [31]:
len(data.columns)

14

In [6]:
cols = list(data.columns)

In [7]:
cols.remove("price")

In [8]:
cols.remove("id")

In [9]:
cols

['country',
 'description',
 'designation',
 'points',
 'province',
 'region_1',
 'region_2',
 'taster_name',
 'taster_twitter_handle',
 'title',
 'variety',
 'winery']

In [10]:
len(data["country"].unique())

46

In [ ]:
pd.get_dummies(data["country"])

In [32]:
d = pd.get_dummies(data[["country","region_1","region_2","province","taster_name","variety"]])
d["points"]= data["points"]
d["id"] = data['id']

In [39]:
X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]

In [41]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
X_train.shape

(175000, 2622)

In [43]:
X_train = X_train.drop(["id"],axis=1)

In [44]:
%%time
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

CPU times: user 40 s, sys: 864 ms, total: 40.9 s
Wall time: 40.9 s


In [45]:
#Predict the response for test dataset
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)

y_pred = clf.predict(X_test)

In [46]:
print(y_test[:10],y_pred[:10])

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: price, dtype: float64 [195.  30.  26.  25.  14.  27.  20.  10.  44.  52.]


In [47]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred

In [48]:
result["price"]=result["price"].apply(int)

In [49]:
result.to_csv('out.csv',index=False)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))